In [2]:
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from datetime import datetime
import os

In [6]:
folder_path = (r"/content/drive/MyDrive/MTech Internship/gujarat_ghi_vis_swir.csv")
df = pd.read_csv(folder_path)
# print(df.head(20))

df['date_time'] = pd.to_datetime(df['date_time'], format='mixed')
df = df[(df['date_time'].dt.time >= pd.to_datetime('7:00').time()) &
(df['date_time'].dt.time <= pd.to_datetime('19:30').time())]

cols = list(df)[1]
# print(cols)

df_for_training = df[cols].astype(float)
# print(df_for_training.head(10))
df_for_training = np.array(df_for_training).reshape(-1,1)
print(df_for_training)

[[3.5333332e-01]
 [1.6586666e+01]
 [7.7518330e+01]
 ...
 [1.6666670e-03]
 [0.0000000e+00]
 [0.0000000e+00]]


In [8]:
scaler = MinMaxScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)
print(df_for_training_scaled[:10])


[[3.15975052e-04]
 [1.48329420e-02]
 [6.93222433e-02]
 [1.56257125e-01]
 [2.51784438e-01]
 [3.41975934e-01]
 [4.14960218e-01]
 [4.81726318e-01]
 [5.33447853e-01]
 [5.86835716e-01]]


In [9]:
n_future = 26*3   # Number of days we want to look into the future based on the past days.
n_past = 26*7  # Number of past days we want to use to predict the future.

In [17]:
# Empty lists to be populated using formatted training data
dataX, dataY = [], []
for i in range(0,len(df_for_training)-n_past-n_future):  # 3 hours  prediction
  a = df_for_training[i:(i+n_past), 0]
  dataX.append(a)
  b = df_for_training[i + n_past:i + n_past + n_future, 0]
  dataY.append(b)
dataX, dataY = np.array(dataX), np.array(dataY)
dataX.shape, dataY.shape

((9230, 182), (9230, 78))

In [16]:
# train test split
test_split=0.2
split_size = int(len(dataX) * (1-test_split)) # this will default to 80% train 20% test
train_windows = dataX[:split_size]
train_labels = df_for_training[:split_size]
test_windows = dataX[split_size:]
test_labels = df_for_training[split_size:]